In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Load an example image
img = cv2.imread('example_image.jpg', cv2.IMREAD_GRAYSCALE)

# Apply global thresholding
_, thresholded = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# Apply Otsu's thresholding
_, otsu_thresholded = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Region-based segmentation using Watershed
# Convert to color for watershed processing
img_color = cv2.imread('example_image.jpg')

# Convert to grayscale and apply Otsu's threshold
gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Remove noise by applying morphological operations
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

# Background area (sure background)
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Foreground area (sure foreground)
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Unknown region (where we don't know if it’s background or foreground)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Label markers
_, markers = cv2.connectedComponents(sure_fg)

# Add one to all labels to distinguish the background (which is 0)
markers = markers + 1

# Mark the unknown regions with zero
markers[unknown == 255] = 0

# Apply the Watershed algorithm
markers = cv2.watershed(img_color, markers)
img_color[markers == -1] = [255, 0, 0]  # Mark boundary with red

# Plot the results for comparison
titles = ['Original Image', 'Global Thresholding', 'Otsu Thresholding', 'Watershed Segmentation']
images = [img, thresholded, otsu_thresholded, img_color]

plt.figure(figsize=(10, 8))
for i in range(4):
    plt.subplot(2, 2, i+1)
    if i == 3:
        plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))  # For color image
    else:
        plt.imshow(images[i], cmap='gray')
    plt.title(titles[i])
    plt.axis('off')

# Show the images
plt.tight_layout()
plt.show()